In [ ]:
import pandas as pd 
import numpy as np
import re
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# !unzip /content/cyberbullying_tweets.csv.zip

In [ ]:
df = pd.read_csv("/content/cyberbullying_tweets.csv")

In [ ]:
df.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [ ]:
df['cyberbullying_type'].value_counts()

religion               7998
age                    7992
gender                 7973
ethnicity              7961
not_cyberbullying      7945
other_cyberbullying    7823
Name: cyberbullying_type, dtype: int64

In [ ]:
len(df)

47692

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47692 entries, 0 to 47691
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tweet_text          47692 non-null  object
 1   cyberbullying_type  47692 non-null  object
dtypes: object(2)
memory usage: 745.3+ KB


In [ ]:
np.sum(df.isnull())

tweet_text            0
cyberbullying_type    0
dtype: int64

In [ ]:
tweet, type = list(df["tweet_text"]), list(df["cyberbullying_type"])

In [ ]:
df.iloc[0][0]

'In other words #katandandre, your food was crapilicious! #mkr'

In [ ]:
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()

df['cyberbullying_type_encoded'] = labelencoder.fit_transform(df['cyberbullying_type'])

In [ ]:
df

,tweet_text,cyberbullying_type,cyberbullying_type_encoded
0,"In other words #katandandre, your food was cra...",not_cyberbullying,3
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,3
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,3
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,3
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,3
...,...,...,...
47687,"Black ppl aren't expected to do anything, depe...",ethnicity,1
47688,Turner did not withhold his disappointment. Tu...,ethnicity,1
47689,I swear to God. This dumb nigger bitch. I have...,ethnicity,1
47690,Yea fuck you RT @therealexel: IF YOURE A NIGGE...,ethnicity,1


In [ ]:
df[['cyberbullying_type', 'cyberbullying_type_encoded']].value_counts()

cyberbullying_type   cyberbullying_type_encoded
religion             5                             7998
age                  0                             7992
gender               2                             7973
ethnicity            1                             7961
not_cyberbullying    3                             7945
other_cyberbullying  4                             7823
dtype: int64

In [ ]:
df['tweet_text'] = df['tweet_text'].str.lower()
df.tail()

,tweet_text,cyberbullying_type,cyberbullying_type_encoded
47687,"black ppl aren't expected to do anything, depe...",ethnicity,1
47688,turner did not withhold his disappointment. tu...,ethnicity,1
47689,i swear to god. this dumb nigger bitch. i have...,ethnicity,1
47690,yea fuck you rt @therealexel: if youre a nigge...,ethnicity,1
47691,bro. u gotta chill rt @chillshrammy: dog fuck ...,ethnicity,1


In [ ]:
stopwordlist = ['a', 'about', 'above', 'after', 'again', 'ain', 'all', 'am', 'an',
             'and','any','are', 'as', 'at', 'be', 'because', 'been', 'before',
             'being', 'below', 'between','both', 'by', 'can', 'd', 'did', 'do',
             'does', 'doing', 'down', 'during', 'each','few', 'for', 'from',
             'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here',
             'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in',
             'into','is', 'it', 'its', 'itself', 'just', 'll', 'm', 'ma',
             'me', 'more', 'most','my', 'myself', 'now', 'o', 'of', 'on', 'once',
             'only', 'or', 'other', 'our', 'ours','ourselves', 'out', 'own', 're',
             's', 'same', 'she', "shes", 'should', "shouldve",'so', 'some', 'such',
             't', 'than', 'that', "thatll", 'the', 'their', 'theirs', 'them',
             'themselves', 'then', 'there', 'these', 'they', 'this', 'those',
             'through', 'to', 'too','under', 'until', 'up', 've', 'very', 'was',
             'we', 'were', 'what', 'when', 'where','which','while', 'who', 'whom',
             'why', 'will', 'with', 'won', 'y', 'you', "youd","youll", "youre",
             "youve", 'your', 'yours', 'yourself', 'yourselves']

In [ ]:
STOPWORDS = set(stopwordlist)

def clean_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df['tweet_text'] = df['tweet_text'].apply(lambda text: clean_stopwords(text))

In [ ]:
import string 
english_puctuations = string.punctuation

def clean_punctuations(text):
  translator = str.maketrans('', '', english_puctuations)
  return text.translate(translator)

df['tweet_text'] = df['tweet_text'].apply(lambda x: clean_punctuations(x))

In [ ]:
df["tweet_text"][20]

'halalcunty biebervalue liamxkiwi greenlinerzjm halalflaws know saudis chased girls burning building httptco8b1aclczn9'

In [ ]:
def clean_repeating_characters(text):
    return re.sub(r'(.)1+', r'1', text)

df['tweet_text'] = df['tweet_text'].apply(lambda x: clean_repeating_characters(x))

In [ ]:
df['tweet_text'][55]

'bullshit mkr mkr215'

In [ ]:
def clean_URLs(text):
  return re.sub(r"((www.[^s]+)|(https\S+))", "", text)

df['tweet_text'] = df['tweet_text'].apply(lambda x : clean_URLs(x))


In [ ]:
df['tweet_text'][280]

'andaasonsan '

In [ ]:
def clean_numeric(text):
  return re.sub('[0-9]+', '', text)

df["tweet_text"] = df['tweet_text'].apply(lambda x : clean_numeric(x))

In [ ]:
df['tweet_text'][223]

'fact  cute boys world  go school sick shit'

In [ ]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer('\w+')
df['tweet_text'] = df["tweet_text"].apply(tokenizer.tokenize)

In [ ]:
df['tweet_text'][24]

['wishing',
 'arena',
 'partner',
 'on',
 'gtgt',
 'really',
 'want',
 'get',
 'pvp',
 'happening']

In [ ]:
import nltk

st = nltk.PorterStemmer()

def text_stemming(text):
  text = [st.stem(word) for word in text]
  return text

df['tweet_text'] = df['tweet_text'].apply(lambda x : text_stemming(x))

In [ ]:
df['tweet_text'][35]

['andythewooki',
 'what',
 'quot',
 'jack',
 'off',
 'one',
 'hand',
 'point',
 'other']

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
lm = nltk.WordNetLemmatizer()

def text_lemmatization(text):
  text = [lm.lemmatize(word) for word in text]
  return text

df['tweet_text'] = df['tweet_text'].apply(lambda x : text_lemmatization(x))

In [ ]:
df['tweet_text'].head()

0             [word, katandandr, food, crapilici, mkr]
1    [aussietv, white, mkr, theblock, imacelebritya...
2    [xochitlsuckkk, classi, whore, red, velvet, cu...
3    [jasongio, meh, p, thank, head, up, but, not, ...
4    [rudhoeenglish, isi, account, pretend, kurdish...
Name: tweet_text, dtype: object

In [ ]:
df

,tweet_text,cyberbullying_type,cyberbullying_type_encoded
0,"[word, katandandr, food, crapilici, mkr]",not_cyberbullying,3
1,"[aussietv, white, mkr, theblock, imacelebritya...",not_cyberbullying,3
2,"[xochitlsuckkk, classi, whore, red, velvet, cu...",not_cyberbullying,3
3,"[jasongio, meh, p, thank, head, up, but, not, ...",not_cyberbullying,3
4,"[rudhoeenglish, isi, account, pretend, kurdish...",not_cyberbullying,3
...,...,...,...
47687,"[black, ppl, arent, expect, anyth, depend, any...",ethnicity,1
47688,"[turner, not, withhold, disappoint, turner, ca...",ethnicity,1
47689,"[swear, god, dumb, nigger, bitch, got, bleach,...",ethnicity,1
47690,"[yea, fuck, rt, therealexel, nigger, fuck, unf...",ethnicity,1


In [ ]:
df[['cyberbullying_type', 'cyberbullying_type_encoded']].value_counts()

cyberbullying_type   cyberbullying_type_encoded
religion             5                             7998
age                  0                             7992
gender               2                             7973
ethnicity            1                             7961
not_cyberbullying    3                             7945
other_cyberbullying  4                             7823
dtype: int64

In [ ]:
df

,tweet_text,cyberbullying_type,cyberbullying_type_encoded
0,"[word, katandandr, food, crapilici, mkr]",not_cyberbullying,3
1,"[aussietv, white, mkr, theblock, imacelebritya...",not_cyberbullying,3
2,"[xochitlsuckkk, classi, whore, red, velvet, cu...",not_cyberbullying,3
3,"[jasongio, meh, p, thank, head, up, but, not, ...",not_cyberbullying,3
4,"[rudhoeenglish, isi, account, pretend, kurdish...",not_cyberbullying,3
...,...,...,...
47687,"[black, ppl, arent, expect, anyth, depend, any...",ethnicity,1
47688,"[turner, not, withhold, disappoint, turner, ca...",ethnicity,1
47689,"[swear, god, dumb, nigger, bitch, got, bleach,...",ethnicity,1
47690,"[yea, fuck, rt, therealexel, nigger, fuck, unf...",ethnicity,1


In [ ]:
df['tweet_text'] = df['tweet_text'].apply(lambda x : " ".join(x))
df

,tweet_text,cyberbullying_type,cyberbullying_type_encoded
0,word katandandr food crapilici mkr,not_cyberbullying,3
1,aussietv white mkr theblock imacelebrityau tod...,not_cyberbullying,3
2,xochitlsuckkk classi whore red velvet cupcak,not_cyberbullying,3
3,jasongio meh p thank head up but not concern a...,not_cyberbullying,3
4,rudhoeenglish isi account pretend kurdish acco...,not_cyberbullying,3
...,...,...,...
47687,black ppl arent expect anyth depend anyth yet ...,ethnicity,1
47688,turner not withhold disappoint turner call cou...,ethnicity,1
47689,swear god dumb nigger bitch got bleach hair re...,ethnicity,1
47690,yea fuck rt therealexel nigger fuck unfollow m...,ethnicity,1


In [ ]:
X,y = df["tweet_text"], df['cyberbullying_type_encoded']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=41)

In [ ]:
X_train

10035    anoth fellow gay plea don t compar revers high...
14737    sinc hard new norm femal still act shi don t n...
1209     thanksgiv drink game classic half tabl no idea...
12259    friday morn workplac joke move gay joke rape j...
45672    mattscorpio ya nigga bf dump huh tough break r...
                               ...                        
31597           peerwork uh one group liter kill peopl nut
41281    racism real world declin must live fuck matrix...
20450    but isi quran noraml muslim quran same but say...
931      nickdwilliam know putin would spend second loo...
34752    got differ girl bachelor show girl w short hai...
Name: tweet_text, Length: 33384, dtype: object

In [ ]:
vectoriser = TfidfVectorizer(ngram_range=(1,2), max_features= 500000)
vectoriser.fit(X_train)
print("No. of feature words: " , len(vectoriser.get_feature_names()))                                                                                               

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


No. of feature words:  316976


In [ ]:
X_train = vectoriser.transform(X_train)
X_test = vectoriser.transform(X_test)


In [ ]:
from sklearn.svm import SVC
svm_model_linear = SVC(kernel = 'linear', C = 1).fit (X_train, y_train)
svm_predictions = svm_model_linear.predict(X_test)
accuracy = svm_model_linear.score(X_test, y_test)
print(accuracy)


0.8286972323175845


In [ ]:
example = np.zeros(100)
example = ["Girl bully’s as well. I’ve 2 sons that were bullied in Jr High. Both were bullied by girls. My older was bullied because he had 4ft long brown hair and a baby face. Younger was bullied cuz he hung around the nerd crowd and was an easy target. I know what u mean though! Peace",]
example = vectoriser.transform(example)
example

<1x316976 sparse matrix of type '<class 'numpy.float64'>'
	with 47 stored elements in Compressed Sparse Row format>

In [ ]:
svm_model_linear.predict(example)

array([0])

In [ ]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(svm_model_linear, open(filename, 'wb'))

In [ ]:
import pickle
filename = 'vectoriser.sav'
pickle.dump(vectoriser, open(filename, 'wb'))